#### Packages installation

In [1]:
!pip install -q "git+https://github.com/panalexeu/horchunk.git"
!pip install -q "git+https://github.com/brandonstarxel/chunking_evaluation.git"
!pip install -q numpy
!pip install -q rich

#### Define chunker wrapper to evaluate over chunking_evaluation framework

In [2]:
from horchunk.chunkers import WindowChunker 
from horchunk.splitters import SentenceSplitter 
from chromadb.utils import embedding_functions
from chunking_evaluation.chunking.base_chunker import BaseChunker

ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2", device='cuda')

class WinChunkerWrapper(BaseChunker):    
    def __init__(self, thresh: float, max_chunk_size: int):
        self.thresh = thresh
        self.max_chunk_size = max_chunk_size
        
    def split_text(self, text: str) -> list[str]:
        chunker = WindowChunker(
            ef,
            thresh=self.thresh,
            max_chunk_size=self.max_chunk_size
        )
        splits = SentenceSplitter(text).__call__()
        chunks = chunker.__call__(splits)

        return [chunk.join() for chunk in chunks]

#### Start evaluation

In [3]:
def parse_res(res: dict) -> dict:
    return dict(
        iou_mean=res['iou_mean'],
        iou_std=res['iou_std'],
        recall_mean=res['recall_mean'],
        recall_std=res['recall_std'],
        precision_omeaga_mean=res['precision_omega_mean'],
        precision_omeaga_std=res['precision_omega_std'],
        precision_mean=res['precision_mean'], 
        precision_std=res['precision_std']
    )

In [4]:
from chunking_evaluation import GeneralEvaluation
from rich import print

chunker = WinChunkerWrapper(thresh=0.72, max_chunk_size=3)
res = GeneralEvaluation().run(chunker, ef)
print(parse_res(res))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4458/4458 [00:40<00:00, 110.85it/s]
⚠️ It looks like you upgraded from a version below 0.6 and could benefit from vacuuming your database. Run chromadb utils vacuum --help for more information.


{
    'iou_mean': np.float64(0.10010845621110019),
    'iou_std': np.float64(0.08304213675576232),
    'recall_mean': np.float64(0.6868400790656709),
    'recall_std': np.float64(0.4132262209206925),
    'precision_omeaga_mean': np.float64(0.4837208428268089),
    'precision_omeaga_std': np.float64(0.19830973720375783),
    'precision_mean': np.float64(0.10279021981351347),
    'precision_std': np.float64(0.08511779997844264)
}

In [5]:
from chunking_evaluation import GeneralEvaluation
from rich import print

chunker = WinChunkerWrapper(thresh=0.72, max_chunk_size=6)
res = GeneralEvaluation().run(chunker, ef)
print(parse_res(res))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4458/4458 [00:35<00:00, 125.15it/s]


{
    'iou_mean': np.float64(0.07457946974175261),
    'iou_std': np.float64(0.06347238660031396),
    'recall_mean': np.float64(0.7494877946643443),
    'recall_std': np.float64(0.4004419189365244),
    'precision_omeaga_mean': np.float64(0.35019504350976166),
    'precision_omeaga_std': np.float64(0.1831203236142467),
    'precision_mean': np.float64(0.0758720256808133),
    'precision_std': np.float64(0.06528374616270594)
}